In [1]:
import pandas as pd
import numpy as np
import time
import math

In [2]:
df=pd.read_csv("data.csv")

In [9]:
# Testing changes
# df1=df[0:10]
# col=["Total Raised","Employees","Revenue"]

In [3]:
#order_df is the copy of data which is used to calculate ranks for each of the columns passed in the colnames variable
#After calculating ranks a new column is added to the order_df with name as colname_rank
#NA values are pushed to the bottom and assigned lowest rank (rank = 1); most relevant are the results with highest rank.

class relevance(object):
    
    def __init__(self, data, searchterm, weights):
        """
        Setting up global variables
        """
        self.starttime=time.time()
        self.order_df = data.copy()
        self.df_colnames = list(self.order_df)
        self.searchterm = searchterm
        self.rank_colname_list = []
        self.weights = weights
        #self.flag = False
        self.mostrelvdf = None
        
    def relevance_sort(self, colnames, popularitycolnames):
        
        #Ignoring search terms with character length less than 2
        if len(self.searchterm.strip())>2:
            #Segregating exact match results from the data to be relevantly ordered.
            #Checking the search term in Company Name and Ticker columns
            self.mostrelvdf = self.order_df[self.order_df['Company Name'].apply(
                lambda x: x.lower()).str.contains(self.searchterm.lower()) | self.order_df['Ticker'].apply(
                lambda x: str(x).lower()).str.contains(self.searchterm.lower())].reset_index()
            
            print(len(self.mostrelvdf))
            
            self.order_df = self.order_df[~self.order_df['Company Name'].apply(
                lambda x: x.lower()).str.contains(self.searchterm.lower())] 
            self.order_df = self.order_df[~self.order_df['Ticker'].apply(
                lambda x: str(x).lower()).str.contains(self.searchterm.lower())].reset_index()
            print(len(self.order_df))
            
        order = self.calculate_rank(self.order_df, colnames, popularitycolnames, self.weights)
        #Calculating mean rank and adding a order column
        self.order_df.insert(0,'order',order )
        
        #print(order_df) #to print order -- for reference
        print("Generated order for ", len(self.order_df), " rows in ", time.time()-self.starttime," seconds")
        
        #Sorting based on order column to get relevant results on top
        self.order_df = self.order_df.sort_values(by='order', ascending = False, na_position = 'last' )
        
        #Concating exact match df and order df
        if self.mostrelvdf is not None:
            self.final_df = self.mostrelvdf.append(self.order_df)[self.df_colnames]
            print(len(self.final_df))
            return(self.final_df)
        else:
            print(len(self.order_df))
            return(self.order_df)[self.df_colnames]

    def calculate_rank(self, order_df, colnames, popcol, weights):
        all_cols=colnames+['pop_rank']
        #Calculate rank for colnames and popcol
        rank_df=order_df[colnames+popcol].apply(lambda x: x.rank(na_option = 'keep' ))
        #Substitue NAs with lowest possible rank = 1
        rank_df.fillna(1, inplace=True)
        #Calculating mean of popularity columns and appending it to rank_df
        rank_df['pop_rank']=rank_df[popcol].mean(axis=1)
        #Multiplying individual columns with their weights
        rank_df[all_cols]=rank_df[all_cols]*weights
        #Calculating mean of all weighted ranks
        order=rank_df[all_cols].mean(axis=1)
        return order

In [4]:
relevance_obj = relevance(df, "uberfan" , [0.8,0.1,0,0,0,0,0.1])

#Testing

#These are quantitative columns 
col=["Total Raised","Employees","Growth Rate","# Active Investors","Revenue","Growth Rate Change"]

#These are popularity based columns, a single rank is calculated oveall for these columns
popcol = [ "Social Growth Rate","Compete Growth Rate","Web Growth Rate", "Facebook Likes", "Twitter Followers"
          , "Facebook Likes Change"]

#col=["Total Raised"]
#sorted_df = relevance_sort(df1,col)
sorted_df = relevance_obj.relevance_sort(col, popcol)
sorted_df.to_csv("out.csv",encoding="utf-8")
#(sorted_df)


1
48484
Generated order for  48484  rows in  2.0706982612609863  seconds
48485


In [73]:
# Questions:
# 1. What do you do about categorical features
# 2. Missing values? default sorting?
# 3. Can the description be used? (jargon distance)
# 4. Multiple rows have the same average, secondary sort column?

# 5. Revenue column has only 8k values, 40k rows are empty 
# 6. Not sure how to handle "Facebook Likes Change"
# 7. Skew caused by a lot of NA values in one column and not many in other columns.. any way to normalize?

sorted_df

,Unnamed: 0,Company ID,Company Name,Company Former Name,Company Also Known As,PBId,Description,Primary Industry Sector,Primary Industry Group,Primary Industry Code,...,Facebook Likes,Facebook Likes Change,Facebook Likes % Change,Majestic Referring Domains,Majestic Referring Domains Change,Majestic Referring Domains % Change,Twitter Followers,Twitter Followers Change,Twitter Followers % Change,PitchBook Link
0,7356,96885-28,Uberfan,NaN,NaN,96885-28,Developer of a social media application for ba...,Information Technology,Software,Social/Platform Software,...,NaN,NaN,NaN,8.0,0.0,0.000000,220.0,0.0,0.000000,NaN
48483,48491,124219-54,(ALTO Real Estate Funds/Western Avenue Capital),NaN,NaN,124219-54,Provider of commercial real estate investment ...,Consumer Products and Services (B2C),Services (Non-Financial),Real Estate Services (B2C),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44279,44287,137515-69,Orion Informatics,NaN,NaN,137515-69,Developer of corporate performance management ...,Information Technology,Software,Business/Productivity Software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44215,44223,98842-42,PAN Capital Holdings,NaN,NaN,98842-42,Operator of a holding company. The company thr...,Consumer Products and Services (B2C),Services (Non-Financial),Real Estate Services (B2C),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44212,44220,135957-88,Panda Patriot,Moxie Patriot,NaN,135957-88,A 829 megawatt gas fired power plant located i...,Energy,"Exploration, Production and Refining",Energy Production,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44210,44218,154183-96,Pangea Solutions,NaN,NaN,154183-96,Developer of network long haul technologies an...,Information Technology,Software,Network Management Software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44209,44217,40676-95,Panhandle Investments,NaN,NaN,40676-95,Owner of royalty interests in certain oil and ...,Energy,"Exploration, Production and Refining",Energy Exploration,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44208,44216,55548-19,Panhandle-Plains Student Loan Center,NaN,PPSLC,55548-19,Provider of financial services to educational ...,Financial Services,Other Financial Services,Specialized Finance,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44207,44215,62114-23,Pannaway Networks,NaN,Pannaway,62114-23,Developer of broadband access provisions. The ...,Information Technology,Communications and Networking,Internet Service Providers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44205,44213,148194-55,PaperEye,NaN,NaN,148194-55,Provider of an online news sharing platform. T...,Consumer Products and Services (B2C),Media,Information Services (B2C),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
